In [1]:
colab = 1
if colab :
  from google.colab import drive
  drive.mount('/content/drive')
  %cd /content/drive/MyDrive/Github/Opti-Project/src/vision_experiments
  !pip install einops

Mounted at /content/drive
/content/drive/MyDrive/Github/Opti-Project/src/vision_experiments
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.0 MB/s eta 0:00:00


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:

import os
import random
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
from functools import partial
from pathlib import Path
from PIL import Image
from train_utils import *
from vit_small import *
from vit import *

In [4]:
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
#32 42 73
set_seed(42)

cuda:0


In [5]:
def get_data(dataset_name, img_size):
  data_dir = './data/' + dataset_name
  batch_size = 64
  train_loader, val_loader, test_loader = get_loaders(dataset_name, data_dir, batch_size=batch_size, img_size=img_size, augment = False)
  return train_loader, val_loader, test_loader

In [6]:
""" criterion = nn.CrossEntropyLoss()
num_classes = 10
optimizer = optim.SGD(net.parameters(), lr=args.lr,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200) """

' criterion = nn.CrossEntropyLoss()\nnum_classes = 10\noptimizer = optim.SGD(net.parameters(), lr=args.lr,\n                      momentum=0.9, weight_decay=5e-4)\nscheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200) '

# Resnet

## Backward

In [9]:
train_loader, val_loader, test_loader = get_data("CIFAR10",64)

Files already downloaded and verified
Files already downloaded and verified
Train dataset: 45000 batch size 64 of dim torch.Size([3, 64, 64])
Validation dataset: 5000 batch size 64 of dim torch.Size([3, 64, 64])
Test dataset: 10000 batch size 64 of dim torch.Size([3, 64, 64])


In [ ]:
nb_epochs = 20
#no pretrained weights
model_name = "resnet18"
model = models.resnet18(norm_layer=partial(nn.BatchNorm2d, track_running_stats=False))
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.05, patience=2, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)
output_path = Path("./visionModels_runs/backward/")

In [ ]:
train_run, val_run, train_run_its, val_run_its, run_path=train_validate_backward(
    device,
    nb_epochs,
    model,
    model_name,
    train_loader,
    val_loader,
    optimizer,
    scheduler,
    output_path,
    gradient_clip_val=0)
eval=evaluate(device, model, test_loader, run_path=run_path)
print("test results")
eval

	Training Epoch 1:   0%|          | 0/704 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
	Training Epoch 1: 100%|█████████▉| 702/704 [00:29<00:00, 24.17it/s, acc=0.516, f1=0.471, iters_per_sec=23.7, loss=1.46]/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
	Valid

Epoch 1 lr: 0.001 Averages : Train Loss: 1.7212 Val Loss: 1.3717 Train Acc: 0.4096 Val Acc: 0.5087 Train F1: 0.3823 Val F1: 0.4748
Best model saved


	Validating: 100%|██████████| 79/79 [00:02<00:00, 29.38it/s, iters_per_sec=29.4, vacc=0.5, vf1=0.396, vloss=1.4]


Epoch 2 lr: 0.001 Averages : Train Loss: 1.2105 Val Loss: 1.1788 Train Acc: 0.5652 Val Acc: 0.5763 Train F1: 0.5436 Val F1: 0.5542
Best model saved


	Validating: 100%|██████████| 79/79 [00:02<00:00, 29.97it/s, iters_per_sec=30, vacc=0.625, vf1=0.433, vloss=1.13]


...

	Validating: 100%|██████████| 79/79 [00:02<00:00, 30.07it/s, iters_per_sec=30.1, vacc=0.25, vf1=0.111, vloss=2.58]


Epoch 20 lr: 6.250000000000001e-09 Averages : Train Loss: 0.0501 Val Loss: 1.1569 Train Acc: 0.9939 Val Acc: 0.6820 Train F1: 0.9928 Val F1: 0.6619


	Evaluating: 100%|██████████| 157/157 [00:05<00:00, 30.42it/s, eval_acc=0.688, eval_f1=0.507, iters_per_sec=30.4]

test results


{'Acc': 0.6780453821656051, 'F1': 0.6588850729583294}

In [ ]:
nb_epochs = 20
#no pretrained weights
model_name = "resnet34"
model = models.resnet34(norm_layer=partial(nn.BatchNorm2d, track_running_stats=False))
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.05, patience=2, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)
output_path = Path("./visionModels_runs/backward/")

In [ ]:
train_run, val_run, train_run_its, val_run_its, run_path=train_validate_backward(
    device,
    nb_epochs,
    model,
    model_name,
    train_loader,
    val_loader,
    optimizer,
    scheduler,
    output_path,
    gradient_clip_val=0)
eval=evaluate(device, model, test_loader, run_path=run_path)
print("test results")
eval

	Validating: 100%|██████████| 79/79 [00:02<00:00, 28.13it/s, iters_per_sec=28.1, vacc=0.25, vf1=0.146, vloss=1.46]


Epoch 1 lr: 0.001 Averages : Train Loss: 1.7721 Val Loss: 1.4706 Train Acc: 0.3747 Val Acc: 0.4555 Train F1: 0.3445 Val F1: 0.4238
Best model saved


	Validating: 100%|██████████| 79/79 [00:02<00:00, 28.84it/s, iters_per_sec=28.8, vacc=0.375, vf1=0.208, vloss=1.26]


Epoch 2 lr: 0.001 Averages : Train Loss: 1.3001 Val Loss: 1.2665 Train Acc: 0.5305 Val Acc: 0.5386 Train F1: 0.5073 Val F1: 0.5082
Best model saved


	Validating: 100%|██████████| 79/79 [00:02<00:00, 28.48it/s, iters_per_sec=28.5, vacc=0.5, vf1=0.367, vloss=1.23]


...

	Validating: 100%|██████████| 79/79 [00:02<00:00, 28.62it/s, iters_per_sec=28.6, vacc=0.625, vf1=0.571, vloss=1.92]


Epoch 20 lr: 6.250000000000001e-09 Averages : Train Loss: 0.1287 Val Loss: 1.2069 Train Acc: 0.9721 Val Acc: 0.6507 Train F1: 0.9686 Val F1: 0.6310


	Evaluating: 100%|██████████| 157/157 [00:05<00:00, 28.05it/s, eval_acc=0.75, eval_f1=0.633, iters_per_sec=28]

test results


{'Acc': 0.6512738853503185, 'F1': 0.6334925478799893}

In [ ]:
nb_epochs = 20
#no pretrained weights
model_name = "resnet50"
model = models.resnet50(norm_layer=partial(nn.BatchNorm2d, track_running_stats=False))
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.05, patience=2, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)
output_path = Path("./visionModels_runs/backward/")

In [ ]:
train_run, val_run, train_run_its, val_run_its, run_path=train_validate_backward(
    device,
    nb_epochs,
    model,
    model_name,
    train_loader,
    val_loader,
    optimizer,
    scheduler,
    output_path,
    gradient_clip_val=0)
eval=evaluate(device, model, test_loader, run_path=run_path)
print("test results")
eval

	Validating: 100%|██████████| 79/79 [00:02<00:00, 26.76it/s, iters_per_sec=26.7, vacc=0.375, vf1=0.333, vloss=1.72]


Epoch 1 lr: 0.001 Averages : Train Loss: 2.1072 Val Loss: 1.7530 Train Acc: 0.2422 Val Acc: 0.3487 Train F1: 0.1962 Val F1: 0.3235
Best model saved


	Validating: 100%|██████████| 79/79 [00:02<00:00, 26.61it/s, iters_per_sec=26.6, vacc=0.625, vf1=0.35, vloss=1.23]


Epoch 2 lr: 0.001 Averages : Train Loss: 1.6661 Val Loss: 1.6654 Train Acc: 0.3908 Val Acc: 0.4193 Train F1: 0.3542 Val F1: 0.3530
Best model saved


	Validating: 100%|██████████| 79/79 [00:02<00:00, 26.71it/s, iters_per_sec=26.7, vacc=0.625, vf1=0.542, vloss=1.21]


...

	Validating: 100%|██████████| 79/79 [00:02<00:00, 26.41it/s, iters_per_sec=26.4, vacc=0.5, vf1=0.396, vloss=0.938]


Epoch 20 lr: 1.2500000000000002e-07 Averages : Train Loss: 0.1433 Val Loss: 1.3437 Train Acc: 0.9619 Val Acc: 0.6222 Train F1: 0.9573 Val F1: 0.6035


	Evaluating: 100%|██████████| 157/157 [00:06<00:00, 25.63it/s, eval_acc=0.688, eval_f1=0.541, iters_per_sec=25.6]

test results


{'Acc': 0.6181329617834395, 'F1': 0.5990396815154153}

In [ ]:
nb_epochs = 20
#no pretrained weights
model_name = "resnet101"
model = models.resnet101(norm_layer=partial(nn.BatchNorm2d, track_running_stats=False))
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.05, patience=2, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)
output_path = Path("./visionModels_runs/backward/")

In [ ]:
train_run, val_run, train_run_its, val_run_its, run_path=train_validate_backward(
    device,
    nb_epochs,
    model,
    model_name,
    train_loader,
    val_loader,
    optimizer,
    scheduler,
    output_path,
    gradient_clip_val=0)
eval=evaluate(device, model, test_loader, run_path=run_path)
print("test results")
eval

	Validating: 100%|██████████| 79/79 [00:03<00:00, 21.88it/s, iters_per_sec=21.9, vacc=0.5, vf1=0.405, vloss=1.49]


Epoch 1 lr: 0.001 Averages : Train Loss: 2.1833 Val Loss: 1.9716 Train Acc: 0.2221 Val Acc: 0.2925 Train F1: 0.1704 Val F1: 0.2577
Best model saved


	Validating: 100%|██████████| 79/79 [00:03<00:00, 22.02it/s, iters_per_sec=22, vacc=0, vf1=0, vloss=2.59]


Epoch 2 lr: 0.001 Averages : Train Loss: 1.8019 Val Loss: 1.6962 Train Acc: 0.3533 Val Acc: 0.3869 Train F1: 0.3172 Val F1: 0.3541
Best model saved


	Validating: 100%|██████████| 79/79 [00:03<00:00, 21.97it/s, iters_per_sec=22, vacc=0.25, vf1=0.19, vloss=1.53]


...

	Validating: 100%|██████████| 79/79 [00:03<00:00, 22.24it/s, iters_per_sec=22.2, vacc=0.5, vf1=0.367, vloss=1.65]


Epoch 20 lr: 6.250000000000001e-09 Averages : Train Loss: 0.6836 Val Loss: 1.3001 Train Acc: 0.7656 Val Acc: 0.5653 Train F1: 0.7489 Val F1: 0.5460


	Evaluating: 100%|██████████| 157/157 [00:07<00:00, 21.72it/s, eval_acc=0.5, eval_f1=0.322, iters_per_sec=21.7]

test results


{'Acc': 0.5673765923566879, 'F1': 0.5481761165958638}

## Forward

In [ ]:
nb_epochs = 20
#no pretrained weights
model_name = "resnet18"
model = models.resnet18(norm_layer=partial(nn.BatchNorm2d, track_running_stats=False))
optimizer =  optim.Adam(model.parameters(),lr=2e-4, eps= 1e-4, weight_decay= 0.0)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.05, patience=2, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)
output_path = Path("./visionModels_runs/forward/")

In [ ]:
train_run, val_run, train_run_its, val_run_its, run_path=train_validate_forward(
    device,
    nb_epochs,
    model,
    model_name,
    train_loader,
    val_loader,
    optimizer,
    scheduler,
    output_path,
    gradient_clip_val=0)
eval=evaluate(device, model, test_loader, run_path=run_path)
print("test results")
eval

	Validating: 100%|██████████| 79/79 [00:02<00:00, 29.65it/s, iters_per_sec=29.6, vacc=0, vf1=0, vloss=6.42]


Epoch 1 lr: 0.0002 Averages : Train Loss: 6.7856 Val Loss: 6.5942 Train Acc: 0.0000 Val Acc: 0.0004 Train F1: 0.0000 Val F1: 0.0002
Best model saved


	Validating: 100%|██████████| 79/79 [00:02<00:00, 29.54it/s, iters_per_sec=29.5, vacc=0.25, vf1=0.0833, vloss=6]


Epoch 2 lr: 0.0002 Averages : Train Loss: 6.4251 Val Loss: 6.2503 Train Acc: 0.0058 Val Acc: 0.0245 Train F1: 0.0025 Val F1: 0.0086
Best model saved


	Validating: 100%|██████████| 79/79 [00:02<00:00, 29.41it/s, iters_per_sec=29.4, vacc=0.125, vf1=0.0444, vloss=5.93]


...

	Validating: 100%|██████████| 79/79 [00:02<00:00, 28.37it/s, iters_per_sec=28.4, vacc=0, vf1=0, vloss=2.89]


Epoch 20 lr: 0.0002 Averages : Train Loss: 2.9194 Val Loss: 2.9026 Train Acc: 0.1544 Val Acc: 0.1505 Train F1: 0.1179 Val F1: 0.1152
Best model saved


	Evaluating: 100%|██████████| 157/157 [00:05<00:00, 29.67it/s, eval_acc=0.188, eval_f1=0.135, iters_per_sec=29.7]

test results


{'Acc': 0.14729299363057324, 'F1': 0.1127873344443108}

In [ ]:
nb_epochs = 20
#no pretrained weights
model_name = "resnet34"
model = models.resnet34(norm_layer=partial(nn.BatchNorm2d, track_running_stats=False))
optimizer =  optim.Adam(model.parameters(),lr=2e-4, eps= 1e-4, weight_decay= 0.0)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.05, patience=2, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)
output_path = Path("./visionModels_runs/forward/")

In [ ]:
train_run, val_run, train_run_its, val_run_its, run_path=train_validate_forward(
    device,
    nb_epochs,
    model,
    model_name,
    train_loader,
    val_loader,
    optimizer,
    scheduler,
    output_path,
    gradient_clip_val=0)
eval=evaluate(device, model, test_loader, run_path=run_path)
print("test results")
eval

	Validating: 100%|██████████| 79/79 [00:03<00:00, 26.10it/s, iters_per_sec=26.1, vacc=0, vf1=0, vloss=6.73]


Epoch 1 lr: 0.0002 Averages : Train Loss: 7.0182 Val Loss: 6.9055 Train Acc: 0.0000 Val Acc: 0.0002 Train F1: 0.0001 Val F1: 0.0002
Best model saved


	Validating: 100%|██████████| 79/79 [00:02<00:00, 26.40it/s, iters_per_sec=26.4, vacc=0, vf1=0, vloss=6.83]


Epoch 2 lr: 0.0002 Averages : Train Loss: 6.7933 Val Loss: 6.6785 Train Acc: 0.0003 Val Acc: 0.0002 Train F1: 0.0003 Val F1: 0.0003
Best model saved


	Validating: 100%|██████████| 79/79 [00:03<00:00, 26.01it/s, iters_per_sec=26, vacc=0, vf1=0, vloss=6.19]


...

	Validating: 100%|██████████| 79/79 [00:02<00:00, 26.53it/s, iters_per_sec=26.5, vacc=0.125, vf1=0.0833, vloss=3.83]


Epoch 20 lr: 0.0002 Averages : Train Loss: 3.7350 Val Loss: 3.7071 Train Acc: 0.0973 Val Acc: 0.1028 Train F1: 0.0504 Val F1: 0.0635
Best model saved


	Evaluating: 100%|██████████| 157/157 [00:05<00:00, 26.87it/s, eval_acc=0.125, eval_f1=0.1, iters_per_sec=26.9]

test results


{'Acc': 0.10469745222929937, 'F1': 0.062349787447855305}

In [ ]:
nb_epochs = 20
#no pretrained weights
model_name = "resnet50"
model = models.resnet50(norm_layer=partial(nn.BatchNorm2d, track_running_stats=False))
optimizer =  optim.Adam(model.parameters(),lr=2e-4, eps= 1e-4, weight_decay= 0.0)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.05, patience=2, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)
output_path = Path("./visionModels_runs/forward/")

In [ ]:
train_run, val_run, train_run_its, val_run_its, run_path=train_validate_forward(
    device,
    nb_epochs,
    model,
    model_name,
    train_loader,
    val_loader,
    optimizer,
    scheduler,
    output_path,
    gradient_clip_val=0)
eval=evaluate(device, model, test_loader, run_path=run_path)
print("test results")
eval

	Validating: 100%|██████████| 79/79 [00:03<00:00, 24.53it/s, iters_per_sec=24.5, vacc=0, vf1=0, vloss=6.56]


Epoch 1 lr: 0.0002 Averages : Train Loss: 6.9754 Val Loss: 6.9424 Train Acc: 0.0011 Val Acc: 0.0026 Train F1: 0.0006 Val F1: 0.0015
Best model saved


	Validating: 100%|██████████| 79/79 [00:03<00:00, 24.80it/s, iters_per_sec=24.8, vacc=0, vf1=0, vloss=6.72]


Epoch 2 lr: 0.0002 Averages : Train Loss: 6.8924 Val Loss: 6.8504 Train Acc: 0.0020 Val Acc: 0.0012 Train F1: 0.0012 Val F1: 0.0006
Best model saved


	Validating: 100%|██████████| 79/79 [00:03<00:00, 25.02it/s, iters_per_sec=25, vacc=0, vf1=0, vloss=6.82]


...

	Validating: 100%|██████████| 79/79 [00:03<00:00, 25.21it/s, iters_per_sec=25.2, vacc=0.125, vf1=0.0741, vloss=6.84]


Epoch 20 lr: 0.0002 Averages : Train Loss: 6.0628 Val Loss: 6.0664 Train Acc: 0.0204 Val Acc: 0.0220 Train F1: 0.0094 Val F1: 0.0100


	Evaluating: 100%|██████████| 157/157 [00:06<00:00, 26.07it/s, eval_acc=0, eval_f1=0, iters_per_sec=26.1]

test results


{'Acc': 0.021795382165605094, 'F1': 0.009366117186021006}

In [ ]:
nb_epochs = 20
#no pretrained weights
model_name = "resnet101"
model = models.resnet101(norm_layer=partial(nn.BatchNorm2d, track_running_stats=False))
optimizer =  optim.Adam(model.parameters(),lr=2e-4, eps= 1e-4, weight_decay= 0.0)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.05, patience=2, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)
output_path = Path("./visionModels_runs/forward/")

In [ ]:
train_run, val_run, train_run_its, val_run_its, run_path=train_validate_forward(
    device,
    nb_epochs,
    model,
    model_name,
    train_loader,
    val_loader,
    optimizer,
    scheduler,
    output_path,
    gradient_clip_val=0)
eval=evaluate(device, model, test_loader, run_path=run_path)
print("test results")
eval

# Alexnet

## backward

In [ ]:
train_loader, val_loader, test_loader = get_data("CIFAR10",64)

Files already downloaded and verified
Files already downloaded and verified
Train dataset: 45000 batch size 64 of dim torch.Size([3, 64, 64])
Validation dataset: 5000 batch size 64 of dim torch.Size([3, 64, 64])
Test dataset: 10000 batch size 64 of dim torch.Size([3, 64, 64])


In [ ]:
nb_epochs = 20
#no pretrained weights
model_name = "alexnet"
args =  "()"
model = models.alexnet()
#model = Cifar10CnnModel()
input_size = 1
output_size = 10
#optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)
#test 1e-3, 1e-4
optimizer =  optim.Adam(model.parameters(),lr=2e-4, eps= 1e-4, weight_decay= 0.0)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.05, patience=2, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)
output_path = Path("./visionModels_runs/backward/")

In [ ]:
train_run, val_run, train_run_its, val_run_its, run_path=train_validate_backward(
    device,
    nb_epochs,
    model,
    model_name+"CIFAR10",
    train_loader,
    val_loader,
    optimizer,
    scheduler,
    output_path,
    gradient_clip_val=0)
eval=evaluate(device, model, test_loader, run_path=run_path)
print("test results")
eval

	Validating: 100%|██████████| 79/79 [00:02<00:00, 31.74it/s, iters_per_sec=31.7, vacc=0.5, vf1=0.452, vloss=1.61]


Epoch 1 lr: 0.0002 Averages : Train Loss: 1.9467 Val Loss: 1.5115 Train Acc: 0.2690 Val Acc: 0.4256 Train F1: 0.2079 Val F1: 0.4041
Best model saved


	Validating: 100%|██████████| 79/79 [00:02<00:00, 32.80it/s, iters_per_sec=32.8, vacc=0.75, vf1=0.611, vloss=0.61]


Epoch 2 lr: 0.0002 Averages : Train Loss: 1.2811 Val Loss: 1.1104 Train Acc: 0.5271 Val Acc: 0.5967 Train F1: 0.4953 Val F1: 0.5658
Best model saved


	Validating: 100%|██████████| 79/79 [00:02<00:00, 33.15it/s, iters_per_sec=33.1, vacc=0.875, vf1=0.867, vloss=0.738]


...

	Validating: 100%|██████████| 79/79 [00:02<00:00, 33.23it/s, iters_per_sec=33.2, vacc=0.75, vf1=0.5, vloss=1.11]


Epoch 20 lr: 1.2500000000000004e-09 Averages : Train Loss: 0.0295 Val Loss: 1.2027 Train Acc: 0.9930 Val Acc: 0.7557 Train F1: 0.9919 Val F1: 0.7355


	Evaluating: 100%|██████████| 157/157 [00:04<00:00, 31.74it/s, eval_acc=0.75, eval_f1=0.621, iters_per_sec=31.7]

test results


{'Acc': 0.7449243630573248, 'F1': 0.7282686416442725}

## forward

In [ ]:
nb_epochs = 20
#no pretrained weights
model_name = "alexnet"
model = models.alexnet()
#model = Cifar10CnnModel()
#optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)
optimizer =  optim.Adam(model.parameters(),lr=2e-4, eps= 1e-4, weight_decay= 0.0)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.05, patience=2, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)
#scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda, last_epoch=-1, verbose='deprecated')

output_path = Path("./visionModels_runs/forward/")

In [ ]:
train_run, val_run, train_run_its, val_run_its, run_path=train_validate_forward(
    device,
    nb_epochs,
    model,
    model_name+"CIFAR10",
    train_loader,
    val_loader,
    optimizer,
    scheduler,
    output_path,
    gradient_clip_val=0)
eval=evaluate(device, model, test_loader, run_path=run_path)
print("test results")
eval

	Validating: 100%|██████████| 79/79 [00:02<00:00, 31.97it/s, iters_per_sec=32, vacc=0, vf1=0, vloss=6.85]


Epoch 1 lr: 0.0002 Averages : Train Loss: 6.8816 Val Loss: 6.8389 Train Acc: 0.0632 Val Acc: 0.0892 Train F1: 0.0223 Val F1: 0.0477
Best model saved


	Validating: 100%|██████████| 79/79 [00:02<00:00, 32.53it/s, iters_per_sec=32.5, vacc=0.125, vf1=0.0408, vloss=5.53]


Epoch 2 lr: 0.0002 Averages : Train Loss: 6.5546 Val Loss: 5.7060 Train Acc: 0.0974 Val Acc: 0.1017 Train F1: 0.0642 Val F1: 0.0665
Best model saved


	Validating: 100%|██████████| 79/79 [00:02<00:00, 32.66it/s, iters_per_sec=32.6, vacc=0, vf1=0, vloss=3.94]


...

	Validating: 100%|██████████| 79/79 [00:02<00:00, 31.24it/s, iters_per_sec=31.2, vacc=0.125, vf1=0.0741, vloss=4.23]


Epoch 20 lr: 1.2500000000000004e-09 Averages : Train Loss: 3.5145 Val Loss: 3.5038 Train Acc: 0.1007 Val Acc: 0.1023 Train F1: 0.0924 Val F1: 0.0929


	Evaluating: 100%|██████████| 157/157 [00:04<00:00, 32.01it/s, eval_acc=0.0625, eval_f1=0.0247, iters_per_sec=32]

test results


{'Acc': 0.09375, 'F1': 0.04978585204785185}

# small Vit

## backward

In [ ]:
train_loader, val_loader, test_loader = get_data("CIFAR10",64)

Files already downloaded and verified
Files already downloaded and verified
Train dataset: 45000 batch size 64 of dim torch.Size([3, 64, 64])
Validation dataset: 5000 batch size 64 of dim torch.Size([3, 64, 64])
Test dataset: 10000 batch size 64 of dim torch.Size([3, 64, 64])


In [ ]:
nb_epochs = 20
#no pretrained weights
model_name = "small_vit"
model = ViT_small(
    image_size = 64,
    patch_size = 4,
    num_classes = 10,
    dim = 512,
    depth = 6,
    heads = 8,
    mlp_dim = 256,
    dropout = 0.1,
    emb_dropout = 0.1)
#optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)
optimizer =  optim.Adam(model.parameters(),lr=2e-4, eps= 1e-4, weight_decay= 0.0)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.05, patience=2, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)
#scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda, last_epoch=-1, verbose='deprecated')
output_path = Path("./visionModels_runs/forward/")

In [ ]:
train_run, val_run, train_run_its, val_run_its, run_path=train_validate_backward(
    device,
    nb_epochs,
    model,
    model_name+"CIFAR10",
    train_loader,
    val_loader,
    optimizer,
    scheduler,
    output_path,
    gradient_clip_val=0)
eval=evaluate(device, model, test_loader, run_path=run_path)
print("test results")
eval


	Validating: 100%|██████████| 79/79 [00:08<00:00,  8.78it/s, iters_per_sec=8.78, vacc=0.5, vf1=0.295, vloss=1.5]


Epoch 1 lr: 0.0002 Averages : Train Loss: 1.5891 Val Loss: 1.3081 Train Acc: 0.4186 Val Acc: 0.5257 Train F1: 0.3786 Val F1: 0.4956
Best model saved


	Validating: 100%|██████████| 79/79 [00:09<00:00,  8.76it/s, iters_per_sec=8.76, vacc=0.625, vf1=0.444, vloss=1.23]


Epoch 2 lr: 0.0002 Averages : Train Loss: 1.2230 Val Loss: 1.2368 Train Acc: 0.5579 Val Acc: 0.5708 Train F1: 0.5278 Val F1: 0.5322
Best model saved


	Validating: 100%|██████████| 79/79 [00:09<00:00,  8.75it/s, iters_per_sec=8.75, vacc=0.875, vf1=0.733, vloss=0.626]


...

	Validating: 100%|██████████| 79/79 [00:09<00:00,  8.71it/s, iters_per_sec=8.71, vacc=0.5, vf1=0.429, vloss=1.23]


Epoch 20 lr: 1.2500000000000004e-09 Averages : Train Loss: 0.2124 Val Loss: 1.0814 Train Acc: 0.9333 Val Acc: 0.6693 Train F1: 0.9262 Val F1: 0.6508


	Evaluating: 100%|██████████| 157/157 [00:17<00:00,  8.98it/s, eval_acc=0.688, eval_f1=0.754, iters_per_sec=8.98]

test results


{'Acc': 0.6816281847133758, 'F1': 0.6605186756219124}

## forward

In [10]:
nb_epochs = 20
#no pretrained weights
model_name = "small_vit"
model = ViT_small(
    image_size = 64,
    patch_size = 4,
    num_classes = 10,
    dim = 512,
    depth = 6,
    heads = 8,
    mlp_dim = 256,
    dropout = 0.1,
    emb_dropout = 0.1)
#optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)
optimizer =  optim.Adam(model.parameters(),lr=2e-4, eps= 1e-4, weight_decay= 0.0)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.05, patience=2, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)
#scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda, last_epoch=-1, verbose='deprecated')
output_path = Path("./visionModels_runs/forward/")

In [ ]:
Wtrain_run, val_run, train_run_its, val_run_its, run_path=train_validate_forward(
    device,
    nb_epochs,
    model,
    model_name+"CIFAR10",
    train_loader,
    val_loader,
    optimizer,
    scheduler,
    output_path,
    gradient_clip_val=0)
eval=evaluate(device, model, test_loader, run_path=run_path)
print("test results")
eval

	Validating: 100%|██████████| 79/79 [00:09<00:00,  8.69it/s, iters_per_sec=8.69, vacc=0.125, vf1=0.0833, vloss=2.44]


Epoch 1 lr: 0.0002 Averages : Train Loss: 2.3438 Val Loss: 2.3000 Train Acc: 0.1042 Val Acc: 0.1104 Train F1: 0.0759 Val F1: 0.0940
Best model saved


	Validating: 100%|██████████| 79/79 [00:09<00:00,  8.63it/s, iters_per_sec=8.63, vacc=0.375, vf1=0.257, vloss=2.11]


Epoch 2 lr: 0.0002 Averages : Train Loss: 2.2859 Val Loss: 2.2613 Train Acc: 0.1283 Val Acc: 0.1477 Train F1: 0.1116 Val F1: 0.1235
Best model saved


	Validating: 100%|██████████| 79/79 [00:09<00:00,  8.67it/s, iters_per_sec=8.67, vacc=0.125, vf1=0.0625, vloss=2.08]


...

	Validating: 100%|██████████| 79/79 [00:09<00:00,  8.67it/s, iters_per_sec=8.67, vacc=0, vf1=0, vloss=2.44]


Epoch 9 lr: 0.0002 Averages : Train Loss: 2.1434 Val Loss: 2.1326 Train Acc: 0.1890 Val Acc: 0.1950 Train F1: 0.1625 Val F1: 0.1663
Best model saved


	Validating: 100%|██████████| 79/79 [00:09<00:00,  8.67it/s, iters_per_sec=8.66, vacc=0.125, vf1=0.0714, vloss=2.37]


Epoch 10 lr: 0.0002 Averages : Train Loss: 2.1321 Val Loss: 2.1242 Train Acc: 0.1906 Val Acc: 0.1917 Train F1: 0.1642 Val F1: 0.1629
Best model saved


	Training Epoch 11:  93%|█████████▎| 655/704 [05:29<00:24,  1.99it/s, acc=0.266, f1=0.246, iters_per_sec=1.99, loss=2.04]

# Vit

In [ ]:
train_loader, val_loader, test_loader = get_data("CIFAR10",64)

## backward

In [ ]:
nb_epochs = 20
#no pretrained weights
model_name = "vit"
model = ViT(
    image_size = 64,
    patch_size = 4,
    num_classes = 10,
    dim = 512,
    depth = 6,
    heads = 8,
    mlp_dim = 512,
    dropout = 0.1,
    emb_dropout = 0.1)
#optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)
optimizer =  optim.Adam(model.parameters(),lr=2e-4, eps= 1e-4, weight_decay= 0.0)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.05, patience=2, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)
#scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda, last_epoch=-1, verbose='deprecated')
output_path = Path("./visionModels_runs/forward/")

In [ ]:
train_run, val_run, train_run_its, val_run_its, run_path=train_validate_backward(
    device,
    nb_epochs,
    model,
    model_name+"CIFAR10",
    train_loader,
    val_loader,
    optimizer,
    scheduler,
    output_path,
    gradient_clip_val=0)
eval=evaluate(device, model, test_loader, run_path=run_path)
print("test results")
eval


## forward

In [ ]:
nb_epochs = 20
#no pretrained weights
model_name = "vit"
model = ViT(
    image_size = 64,
    patch_size = 4,
    num_classes = 10,
    dim = 512,
    depth = 6,
    heads = 8,
    mlp_dim = 512,
    dropout = 0.1,
    emb_dropout = 0.1)
#optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)
optimizer =  optim.Adam(model.parameters(),lr=2e-4, eps= 1e-4, weight_decay= 0.0)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.05, patience=2, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)
#scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda, last_epoch=-1, verbose='deprecated')
output_path = Path("./visionModels_runs/forward/")

In [ ]:
train_run, val_run, train_run_its, val_run_its, run_path=train_validate_forward(
    device,
    nb_epochs,
    model,
    model_name+"CIFAR10",
    train_loader,
    val_loader,
    optimizer,
    scheduler,
    output_path,
    gradient_clip_val=0)
eval=evaluate(device, model, test_loader, run_path=run_path)
print("test results")
eval

# VGG

In [ ]:
train_loader, val_loader, test_loader = get_data("CIFAR10",64)

## backward

In [ ]:
nb_epochs = 20
#no pretrained weights
model_name = "vgg11"
model = models.vgg11()
#optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)
optimizer =  optim.Adam(model.parameters(),lr=2e-4, eps= 1e-4, weight_decay= 0.0)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.05, patience=2, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)
#scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda, last_epoch=-1, verbose='deprecated')
output_path = Path("./visionModels_runs/forward/")

In [ ]:
train_run, val_run, train_run_its, val_run_its, run_path=train_validate_backward(
    device,
    nb_epochs,
    model,
    model_name+"CIFAR10",
    train_loader,
    val_loader,
    optimizer,
    scheduler,
    output_path,
    gradient_clip_val=0)
eval=evaluate(device, model, test_loader, run_path=run_path)
print("test results")
eval


## forward

In [ ]:
nb_epochs = 20
#no pretrained weights
model_name = "vgg11"
model = models.vgg11()
#optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)
optimizer =  optim.Adam(model.parameters(),lr=2e-4, eps= 1e-4, weight_decay= 0.0)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.05, patience=2, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)
#scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda, last_epoch=-1, verbose='deprecated')
output_path = Path("./visionModels_runs/forward/")

In [ ]:
train_run, val_run, train_run_its, val_run_its, run_path=train_validate_forward(
    device,
    nb_epochs,
    model,
    model_name+"CIFAR10",
    train_loader,
    val_loader,
    optimizer,
    scheduler,
    output_path,
    gradient_clip_val=0)
eval=evaluate(device, model, test_loader, run_path=run_path)
print("test results")
eval